In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

%matplotlib inline

In [47]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [2]:
df = pd.read_csv('energydata_complete.csv')

In [3]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


**Let's normalize our data**

In [15]:
scaler = MinMaxScaler()
le = LabelEncoder()

df['date'] = le.fit_transform(df['date'])

normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
featured_df = df['T2'].values.reshape(-1,1)
heating_target = normalised_df['T6'].values.reshape(-1,1)

**Let us split our dataset into the training and testing dataset**

In [16]:
x_train, x_test, y_train, y_test = train_test_split(featured_df, 
                                                    heating_target, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [17]:
linear_model = LinearRegression()

**Fit the model to the training dataset**

In [18]:
linear_model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

**Obtain predictions**

In [25]:
predicted_values = linear_model.predict(x_test)
model = linear_model.fit(x_train, y_train)

**R^2 value**

In [31]:
r2 = r2_score(y_test, predicted_values)
round(r2.round(2))

1.0

**Mean Absolute Error**

In [22]:
mae = mean_absolute_error(y_test, predicted_values)
print(mae.round(2))

0.08


**Residual Sum of Squares**

In [23]:
rss = np.sum(np.square(y_test - predicted_values))
print(rss.round(2))

66.12


**Root Mean Squared Error**

In [24]:
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print(rmse.round(3))

0.106


**Coefficient of determination**

In [26]:
coef = model.coef_
print(coef.round(2))

[[0.06]]


**Feature weights**

In [ ]:
#weights = pd.DataFrame(model.coef_, x_train, columns=['Weights'])

**Ridge regression**

In [40]:
ridge_reg = Ridge(alpha=0.4)
ridge_model = ridge_reg.fit(x_train, y_train)
ridge_predicted = ridge_reg.predict(x_test)

In [41]:
rmse_ridge = np.sqrt(mean_squared_error(y_test, ridge_predicted))
print(rmse_ridge.round(3))

0.106


In [42]:
print(rmse.round(3))
#The RMSE value didn't change

0.106


**Lasso regression**

In [43]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
lasso_predicted = lasso_reg.predict(x_test)

In [44]:
rmse_lasso = np.sqrt(mean_squared_error(y_test, lasso_predicted))
print(rmse_lasso.round(3))

0.106


In [46]:
lasso_reg.coef_

array([0.06456655])